In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from my_functions import *
from visual_inspection import nanomaggie_to_flux
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [ ]:
%matplotlib inline

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67

In [ ]:
import glob

# name = 'QSO_contaminants'
name = 'QSO_LAES_2'

filename = f'/home/alberto/almacen/Source_cats/{name}/'
files = glob.glob(filename + 'data*')
files.sort()
fi = []

how_many = 99999999999
for i, name in enumerate(files):
    if i == how_many: break
    fi.append(pd.read_csv(name))

data_qso = pd.concat(fi, axis=0, ignore_index=True)
L_lya_mock = data_qso['L_lya']
L_NV_mock = data_qso['L_NV']
z_mock = data_qso['z']
EW_lya = data_qso['EW0']

pm_flx_mock = data_qso.to_numpy()[:, 1: 60 + 1].T
mag_mock = flux_to_mag(pm_flx_mock[-2], w_central[-2])
print(len(mag_mock))

In [ ]:
area_obs = 200
z_min, z_max = 2, 4
r_min, r_max = 16, 24

model = pd.read_csv('../LAEs/MyMocks/csv/PD2016-QSO_LF.csv')
counts_model_2D = model.to_numpy()[:-1, 1:-1].astype(float) * 1e-4 * area_obs
r_yy = np.arange(15.75, 24.25, 0.5)
z_xx = np.arange(0.5, 6, 1)
f_counts = interp2d(z_xx, r_yy, counts_model_2D)

r_x_Arr = np.linspace(r_min, r_max, 1000)
z_x_Arr = np.linspace(z_min, z_max, 1000)
dr = r_x_Arr[1] - r_x_Arr[0]
dz = z_x_Arr[1] - z_x_Arr[0]
counts_grid = f_counts(z_x_Arr, r_x_Arr)

N_src = np.sum(counts_grid) * dr * dz * 2
print(N_src)

In [ ]:
mock_mask = (z_mock > z_min) & (z_mock <= z_max)\
    & (mag_mock < 24) & (mag_mock > 16)
print(sum(mock_mask))

fig, ax = plt.subplots(figsize=(6, 4))

bins = np.arange(42, 47, 0.05)
bin_w = bins[1] - bins[0]
volume = z_volume(z_min, z_max, area_obs)
h = ax.hist(L_lya_mock[mock_mask], bins,
            weights=np.full(L_lya_mock[mock_mask].shape,
                            (volume * bin_w) ** -1),
            histtype='step', label=r'QSO mock Ly$\alpha$',
            linewidth=2)
ax.hist(np.log10(10**L_lya_mock + 10**L_NV_mock)[mock_mask], bins,
        weights=np.full(L_lya_mock[mock_mask].shape, (volume * bin_w) ** -1),
        histtype='step', label=r'QSO mock Ly$\alpha$ + NV',
        linewidth=2)

np.save('tmp/hist_mock', h[0])
np.save('tmp/bins_mock', h[1])

# Plot the reference LF curves
Lx = np.logspace(42, 46, 10000)
phistar1 = 3.33e-6
Lstar1 = 10 ** 44.65
alpha1 = -1.35

Phi_center = schechter(
    Lx, phistar1, Lstar1, alpha1
) * Lx * np.log(10)
ax.plot(
    np.log10(Lx), Phi_center, ls='-.', alpha=0.7,
    label=r'Spinoso et al. (2020) $2.25<z<3.24$', zorder=51,
    color='C7'
)

phistar1 = 10 ** -5.85
Lstar1 = 10 ** 44.6
alpha1 = -1.2
Phi_center = schechter(
    Lx, phistar1, Lstar1, alpha1
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='--', alpha=0.7,
    label='Zhang et al. (2021) $2.0<z<3.5$', zorder=50,
    color='C8'
)

ax.set_yscale('log')
ax.set_ylim(1e-7, 1e-3)
ax.set_xlim(42, 46)
ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$', fontsize=18)
ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\,\Delta\log_{10} L^{-1}$]', fontsize=18)
ax.legend(fontsize=12)
ax.tick_params(direction='in', which='both', labelsize=18)
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.savefig('figures/QSO_mock_Lya_LF.pdf',
            bbox_inches='tight', facecolor='w', edgecolor='w')
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

bins = np.linspace(0, 4.5, 100)
ax.hist(z_mock[mock_mask], bins, histtype='step',
        weights=np.full(z_mock[mock_mask].shape, (bins[1] - bins[0])**-1))
ax.plot(z_xx, np.sum(counts_model_2D, axis=0), c='C1')

ax.set_xlabel('z mock')
ax.set_ylabel(r'$N\cdot(\Delta z\cdot)^{-1}$')
ax.set_yscale('log')

plt.show()

fig, ax = plt.subplots(figsize=(6, 4))

bins = np.linspace(16, 25, 100)
bin_w = bins[1] - bins[0]
ax.hist(mag_mock[mock_mask], bins, weights=np.full(
    mag_mock[mock_mask].shape, (bin_w)**-1), label='Mock')
ax.plot(r_yy, np.sum(counts_model_2D[:, :-2] / 2, axis=1), c='C1')
ax.set_ylabel(r'$N\cdot(\Delta r\cdot)^{-1}$')
ax.set_xlabel('r')
ax.legend()
ax.set_yscale('log')

plt.show()

fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(mag_mock[mock_mask], L_lya_mock[mock_mask],
        ls='', marker='.', markersize=2)
ax.set_ylabel('L_lya')
ax.set_xlabel('r')

plt.show()


In [ ]:
filename_pm_DR16 = ('csv/J-SPECTRA_QSO_Superset_DR16_v2.csv')
pm_SEDs_DR16 = pd.read_csv(filename_pm_DR16)

In [ ]:
Lya_fts = pd.read_csv('csv/Lya_fts_DR16_v2.csv')

In [ ]:
plt.hist(Lya_fts['Lya_z'], np.linspace(0, 5, 1000))

In [ ]:
def load_QSO_prior_mock():
    filename = ('csv/J-SPECTRA_QSO_Superset_DR16_v2.csv')

    def format_string4(x): return '{:04d}'.format(int(x))
    def format_string5(x): return '{:05d}'.format(int(x))
    convert_dict = {
        122: format_string4,
        123: format_string5,
        124: format_string4
    }
    plate_mjd_fiber = pd.read_csv(
        filename, sep=',', usecols=[61, 62, 63],
        converters=convert_dict
    ).to_numpy().T

    plate_mjd_fiber = plate_mjd_fiber[np.array([1, 0, 2])]

    return plate_mjd_fiber

pm_flx = pm_SEDs_DR16.to_numpy()[:, 1:61].T
plate_mjd_fiber = load_QSO_prior_mock()
plate = plate_mjd_fiber[0].astype(int)
mjd = plate_mjd_fiber[1].astype(int)
fiber = plate_mjd_fiber[2].astype(int)
correct_dir = '/home/alberto/cosmos/LAEs/MyMocks/csv/QSO_mock_correct_files'
Lya_fts = pd.read_csv('csv/Lya_fts_DR16_v2.csv')
z_Arr = Lya_fts['Lya_z'].to_numpy()
z_Arr[z_Arr == 0] = -1
lya_NB_Arr = NB_z(z_Arr)

F_lya = np.array(Lya_fts['LyaF']) * 1e-17
F_lya_err = np.array(Lya_fts['LyaF_err']) * 1e-17
dL = cosmo.luminosity_distance(z_Arr).to(u.cm).value
L_lya = np.log10(F_lya * 4*np.pi * dL ** 2)

F_NV = np.array(Lya_fts['NVF']) * 1e-17
L_NV = np.log10(F_NV * 4*np.pi * dL ** 2)

N_src = len(L_lya)

In [ ]:
def contour_curve(xx, yy, bins_xx, bins_yy):
    Z, x, y = np.histogram2d(xx, yy, bins=(bins_xx, bins_yy))

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * (y[1:] + y[:-1])
    x_centers = 0.5 * (x[1:] + x[:-1])

    N_bins = 10000

    H_Arr = np.linspace(H_min, H_max, N_bins)[::-1]

    fact_up_Arr = np.zeros(N_bins)

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):

        mask = Z > H_Arr[iii]

        fact_up_Arr[iii] = np.sum(Z[mask]) / TOTAL_H

    H_value_68 = np.interp(0.683, fact_up_Arr, H_Arr)  # 1sigma
    H_value_95 = np.interp(0.954, fact_up_Arr, H_Arr)  # 2sigma
    H_value_90 = np.interp(0.900, fact_up_Arr, H_Arr)  # 2sigma
    H_value_99 = np.interp(0.997, fact_up_Arr, H_Arr)  # 2sigma
    # levels = [H_value_99]
    levels = [H_value_95, H_value_68]

    return x_centers, y_centers, Z.T, levels


fig, ax = plt.subplots(figsize=(6, 4))

sdss_mask = (F_lya / F_lya_err > 5) & (F_lya_err > 0)

c_xx = flux_to_mag(pm_flx[-2], w_central[-2])[sdss_mask]
c_yy = L_lya[sdss_mask]
xx_bins = np.linspace(16, 24, 100)
yy_bins = np.linspace(40, 50, 100)
ax.plot(c_xx, c_yy, ls='', marker='.', markersize=1, c='lightseagreen', rasterized=True)
xx, yy, Z, levels = contour_curve(
    c_xx, c_yy, xx_bins, yy_bins)
ax.contour(xx, yy, Z, levels, colors='k', linewidths=2, zorder=100)
ax.contour(xx, yy, Z, levels, colors='lightseagreen', linewidths=1, zorder=101)

c_xx = mag_mock
c_yy = L_lya_mock
xx_bins = np.linspace(16, 24, 100)
yy_bins = np.linspace(40, 50, 100)
ax.plot(c_xx, c_yy, ls='', marker='.', markersize=1, c='coral', rasterized=True)
xx, yy, Z, levels = contour_curve(
    c_xx, c_yy, xx_bins, yy_bins)
ax.contour(xx, yy, Z, levels, colors='k', linewidths=2, zorder=200)
ax.contour(xx, yy, Z, levels, colors='coral', linewidths=1, zorder=201)

# Dummy points for the legend
ax.scatter(0, 0, color='coral', label='QSO mock')
ax.scatter(0, 0, color='lightseagreen', label='SDSS DR16Q')

ax.set_ylabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$', fontsize=18)
ax.set_xlabel('$r$ [magAB]', fontsize=18)
ax.set_ylim(41, 47)
ax.set_xlim(16, 24)
ax.legend(fontsize=16)
ax.tick_params(direction='in', which='both', labelsize=18)
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
ax.set_axisbelow(False)

fig.savefig('figures/Llya_vs_rSDSS_mock.pdf',
            bbox_inches='tight', facecolor='w', edgecolor='w')
plt.show()

In [ ]:
from minijpas_LF_and_puricomp import nb_or_3fm_cont

pm_err = np.ones_like(pm_flx)
where_bad_flx = ~np.isfinite(pm_flx)
pm_flx[where_bad_flx] = 0.
pm_err[where_bad_flx] = 9999999999.

cont_est_lya, cont_err_lya, cont_est_other, cont_err_other =\
        nb_or_3fm_cont(pm_flx, np.ones_like(pm_flx), 'nb')

In [ ]:
EW_nb_Arr, EW_nb_e, L_Arr, L_e_Arr, flambda, flambda_e = EW_L_NB(
    pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_NB_Arr, N_nb=0
)

In [ ]:
def contour_curve(xx, yy, bins_xx, bins_yy):
    Z, x, y = np.histogram2d(xx, yy, bins=(bins_xx, bins_yy))

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * ( y[1:] + y[:-1] )
    x_centers = 0.5 * ( x[1:] + x[:-1] )

    N_bins = 10000

    H_Arr = np.linspace(H_min , H_max , N_bins)[::-1]

    fact_up_Arr = np.zeros( N_bins )

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):

        mask = Z > H_Arr[iii]

        fact_up_Arr[iii] = np.sum(Z[ mask ]) / TOTAL_H

    H_value_68 = np.interp(0.683, fact_up_Arr, H_Arr) # 1sigma
    H_value_95 = np.interp(0.954, fact_up_Arr, H_Arr) # 2sigma
    H_value_90 = np.interp(0.900, fact_up_Arr, H_Arr) # 2sigma
    H_value_99 = np.interp(0.997, fact_up_Arr, H_Arr) # 2sigma
    # levels = [H_value_99]
    levels = [H_value_95, H_value_68]

    return x_centers, y_centers, Z.T, levels

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

mask = ((lya_NB_Arr > 1) & (L_lya > 0)
        & (Lya_fts['LyaF'] / Lya_fts['LyaF_err'] > 10)
        & (Lya_fts['NVF'] / Lya_fts['NVF_err'] > 10)
        & (Lya_fts['Lya_z'] > 1))
        # & (Lya_fts['LyaEW'] / (1 + Lya_fts['Lya_z']) > 30))

plot_xx = (Lya_fts['LyaEW'] / (1 + Lya_fts['Lya_z']))[mask]
plot_yy = (L_Arr - L_lya)[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=2, rasterized=True)
bins_xx = np.logspace(0, 4, 100)
bins_yy = np.linspace(-3, 3, 100)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='k', zorder=98)

plot_yy = (L_Arr - np.log10(10**L_lya + 10**L_NV))[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=2, rasterized=True)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='dimgray', zorder=99)

ax.axhline(0, ls='-', c='k')
ax.set_xlim(1, 1e4)
ax.set_ylim(-2, 2)
ax.set_xscale('log')
ax.set_ylabel('$\Delta L$')
ax.set_xlabel(r'Ly$\alpha$ EW$_0$ (SDSS)')

plt.show()

fig, ax = plt.subplots(figsize=(7, 5))

plot_xx = (EW_nb_Arr)[mask]
plot_yy = (L_Arr - L_lya)[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=2, rasterized=True)
bins_xx = np.logspace(0, 4, 100)
bins_yy = np.linspace(-3, 3, 100)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='k', zorder=98)

plot_yy = (L_Arr - np.log10(10**L_lya + 10**L_NV))[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=2, rasterized=True)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='dimgray', zorder=99)

ax.axhline(0, ls='-', c='k')
ax.set_xlim(1, 1e4)
ax.set_ylim(-2, 2)
ax.set_xscale('log')
ax.set_ylabel('$\Delta L$')
ax.set_xlabel(r'Ly$\alpha$ EW$_0$ (Measured)')

plt.show()

fig, ax = plt.subplots(figsize=(7, 5))

plot_xx = (L_lya)[mask]
plot_yy = (L_Arr - L_lya)[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=2, rasterized=True)
bins_xx = np.linspace(42.5, 47, 100)
bins_yy = np.linspace(-3, 3, 100)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='k', zorder=98)

plot_yy = (L_Arr - np.log10(10**L_lya + 10**L_NV))[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=2, rasterized=True)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='dimgray', zorder=99)

ax.axhline(0, ls='-', c='k')
ax.set_xlim(42, 47)
ax.set_ylim(-2, 2)
ax.set_ylabel('$\Delta L$')
ax.set_xlabel(r'Ly$\alpha$ L (SDSS)')

plt.show()

fig, ax = plt.subplots(figsize=(7, 5))

plot_xx = (L_Arr)[mask]
plot_yy = (L_Arr - L_lya)[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=2, rasterized=True)
bins_xx = np.linspace(42.5, 47, 100)
bins_yy = np.linspace(-3, 3, 100)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='k', zorder=98)

plot_yy = (L_Arr - np.log10(10**L_lya + 10**L_NV))[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=2, rasterized=True)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='dimgray', zorder=99)

ax.axhline(0, ls='-', c='k')
ax.set_xlim(42, 47)
ax.set_ylim(-2, 2)
ax.set_ylabel('$\Delta L$')
ax.set_xlabel(r'Ly$\alpha$ L (Measured)')

plt.show()

In [ ]:
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

fig, ax = plt.subplots(figsize=(6, 3))

bins = np.linspace(1, 250, 100)
ax.hist(Lya_fts['LyaEW'] / (1 + Lya_fts['Lya_z']), bins, histtype='step',
        label='Lya EW0')
ax.hist(Lya_fts['NVEW'] / (1 + Lya_fts['Lya_z']), bins, histtype='step',
        label='NV EW0')
ax.set_yscale('log')
ax.legend()
ax.set_xlabel('EW0')

plt.show()

mag = flux_to_mag(pm_flx[-2], w_central[-2])

fig, ax = plt.subplots(figsize=(5, 4))

plot_xx = mag[mask]
plot_yy = (np.log10(Lya_fts['LyaEW']) - np.log10(Lya_fts['NVEW']))[mask]
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=1)
bins_xx = np.linspace(16, 24, 100)
bins_yy = np.linspace(-3, 4, 100)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='k', zorder=98)
ax.axhline(0, c='k')
ax.set_ylim(-2, 2)
ax.set_xlim(17, 23)
ax.set_xlabel('r')
ax.set_ylabel('$\log$ EW_Lya / EW_NV')

plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

dL = cosmo.luminosity_distance(Lya_fts['Lya_z'].to_numpy()[mask]).to(u.cm).value
plot_xx = mag[mask]
plot_yy = (Lya_fts['LyaF'][mask] * 1e-17 * (4 * np.pi * dL**2))
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=1)
bins_xx = np.linspace(16, 24, 100)
bins_yy = np.logspace(41, 48, 100)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='k', zorder=98)
ax.axhline(0, c='k')
ax.set_ylim(1e42, 1e47)
ax.set_xlim(17, 24)
ax.set_xlabel('r')
ax.set_ylabel('F_Lya')
ax.set_yscale('log')

plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

plot_xx = (Lya_fts['LyaEW'] + Lya_fts['NVEW']) / (1 + Lya_fts['Lya_z'])
plot_yy = np.log10(Lya_fts['LyaEW']) - np.log10(Lya_fts['NVEW'])
ax.plot(plot_xx, plot_yy, ls='', marker='.', markersize=1)
bins_xx = np.logspace(0, 3, 100)
bins_yy = np.linspace(-3, 4, 100)
xx, yy, Z, levels = contour_curve(plot_xx, plot_yy, bins_xx, bins_yy)
ax.contour(xx, yy, Z, levels, colors='k', zorder=98)
ax.axhline(0, c='k')
ax.set_ylim(-3, 4)
ax.set_xlim(1e0, 1e3)
ax.set_xscale('log')
ax.set_xlabel('Lya + NV EW0')
ax.set_ylabel('$\log$ EW_Lya / EW_NV')

plt.show()

In [ ]:
selected = np.random.permutation(
    np.where(
        (lya_NB_Arr > 1) & (L_lya > 0) & mask
    )[0]
)
print(f'Selected = {len(selected)}\n')

fits_dir = '/home/alberto/almacen/SDSS_spectra_fits/DR16/QSO'

pm_flx[0, :] = 0
pm_flx[-4, :] = 0
pm_err[0, :] = 0
pm_err[-4, :] = 0

w_lya = 1215.67

for i, src in enumerate(selected):
    if i == 0: break
    spec_name = f'spec-{plate[src]:04d}-{mjd[src]:05d}-{fiber[src]:04d}.fits'
    spzline = Table.read(f'{fits_dir}/{spec_name}', hdu=3, format='fits')

    print(spec_name)
    L_lya_NV = np.log10(10**L_lya[src] + 10**L_NV[src])
    print(f'L_lya = {L_lya[src]:0.2f}, L_NV = {L_NV[src]:0.2f}, L_NV + L_lya = {L_lya_NV:0.2f}')
    print(f'Lya/NV = {10**(L_lya[src] - L_NV[src]):0.2f}')
    print(f'L_Arr = {L_Arr[src]:0.2f}')
    print(f'EW_lya = {Lya_fts["LyaEW"][src]:0.2f}±{np.abs(Lya_fts["LyaEW_err"][src]):0.2f}')
    print(f'EW_NV = {Lya_fts["NVEW"][src]:0.2f}±{np.abs(Lya_fts["NVFEW_err"][src]):0.2f}')
    print(f'EW_Arr = {EW_nb_Arr[src]:0.2f}')
    print(f'Lya_z = {Lya_fts["Lya_z"][src]:0.2f}')

    fig = plt.figure(figsize=(10, 4))
    ax = plot_JPAS_source(pm_flx[:, src], 0 * pm_err[:, src])
    ax.axhline(0, c='k')
    ax.axvline(w_lya * (1 + Lya_fts['Lya_z'][src]), ls='--', c='r')

    spec = Table.read(f'{fits_dir}/{spec_name}', hdu=1, format='fits')
    g_band = Table.read(f'{fits_dir}/{spec_name}', hdu=2, format='fits')['SPECTROFLUX']
    g_band = nanomaggie_to_flux(np.array(g_band)[0][1], 4750)
    norm = pm_flx[-3, src] / g_band
    spec_model = spec['MODEL'] * norm * 1e-17
    spec_flx = spec['FLUX'] * norm * 1e-17
    spec_w = 10 ** spec['LOGLAM']

    ax.plot(spec_w, spec_flx, c='dimgray', zorder=-100, alpha=0.7)
    ax.plot(spec_w, spec_model, c='orange', zorder=-99, alpha=0.7)

    ax.set_xlim(3500, 6000)

    plt.show()